In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import os
import random
import pickle
import time
import torch


########################################################################################################################
# Helper Functions
########################################################################################################################

def count_parameters(model):
	return sum(p.numel() for p in model.parameters() if p.requires_grad)


def Euclidian_dist(user_mat, item_mat):
    A = (user_mat ** 2).sum(1, keepdim=True)
    B = (item_mat ** 2).sum(1, keepdim=True)
    
    AB = -2 * torch.matmul(user_mat, item_mat.T)
    
    return torch.sqrt(A + AB + B.T)   


def to_np(x):
	return x.data.cpu().numpy()


def dict_set(base_dict, user_id, item_id, val):
	if user_id in base_dict:
		base_dict[user_id][item_id] = val
	else:
		base_dict[user_id] = {item_id: val}


def is_visited(base_dict, user_id, item_id):
	if user_id in base_dict and item_id in base_dict[user_id]:
		return True
	else:
		return False


def save_pickle(path, filename, obj):
	with open(path + filename, 'wb') as f:
		pickle.dump(obj, f)


def load_pickle(path, filename):
	with open(path + filename, 'rb') as f:
		obj = pickle.load(f)

	return obj


def list_to_dict(base_list):
	result = {}
	for user_id, item_id, value in base_list:
		dict_set(result, user_id, item_id, value)
	
	return result


def dict_to_list(base_dict):
	result = []

	for user_id in base_dict:
		for item_id in base_dict[user_id]:
			result.append((user_id, item_id, 1))
	
	return result
	

def turn_it_to_boolean(yes_or_no):
	if yes_or_no == 'yes':
		return True
	elif yes_or_no == 'no':
		return False
	else:
		assert False


########################################################################################################################
# For data load
########################################################################################################################


def get_count_dict(f, spliter="\t"):

	user_count_dict, item_count_dict = {}, {}

	for line in f.readlines():
		tmp = line.split(spliter)

		if len(tmp) == 4:
			user, item, rating, timestamp = tmp
		else:
			user, item, rating = tmp

		if user in user_count_dict:
			user_count_dict[user] += 1
		else: 
			user_count_dict[user] = 1

		if item in item_count_dict:
			item_count_dict[item] += 1
		else: 
			item_count_dict[item] = 1

	return user_count_dict, item_count_dict


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F 

import numpy as np

# Dataset for implicit feedback
class implicit_CF_dataset(data.Dataset):

    def __init__(self, user_count, item_count, rating_mat, interactions, num_ns):
        """
        Parameters
        ----------
        user_count : int
            num. users
        item_count : int
            num. items
        rating_mat : dict
            user-item rating matrix
        interactions : list
            total train interactions, each instance has a form of (user, item, 1)
        num_ns : int
            num. negative samples
        """
        super(implicit_CF_dataset, self).__init__()
        
        self.user_count = user_count
        self.item_count = item_count
        self.rating_mat = rating_mat
        self.num_ns = num_ns
        self.interactions = interactions
        self.train_arr = []
        self.negative_sampling()
        

    def negative_sampling(self):
        """conduct the negative sampling
        """      
        
        sample_list = np.random.choice(list(range(self.item_count)), size = 10 * len(self.interactions) * self.num_ns)
        print(len(sample_list))
        
        sample_idx = 0
        for user, pos_item, _ in self.interactions:
            ns_count = 0
            
            while True:
                neg_item = sample_list[sample_idx]
                if not is_visited(self.rating_mat, user, neg_item):
                    self.train_arr.append((user, pos_item, neg_item))
                    sample_idx += 1
                    ns_count += 1
                    if ns_count == self.num_ns:
                        break
                        
                sample_idx += 1
    

    def __len__(self):
        return len(self.interactions) * self.num_ns
        

    def __getitem__(self, idx):
        return self.train_arr[idx]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from random import randint
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import datetime
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import warnings
import pandas as pd
import torch.nn.functional as F
from copy import deepcopy



#ratings = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Data-final/ratings_data.csv")
#ratings = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data.csv")
#ratings = ratings[['userId', 'productId', 'rating']].values

#print(len(np.unique(ratings[:, 0])), len(np.unique(ratings[:, 1])))
#train, test = train_test_split(ratings, test_size=0.2)  # spilt_rating_dat(ratings)
#train = np.loadtxt("/content/drive/MyDrive/Copy of Epinions/Data-final/user_ratings_final_train.txt")
#test = np.loadtxt("/content/drive/MyDrive/Copy of Epinions/Data-final/user_ratings_final_test.txt")
#train = np.loadtxt("/content/drive/MyDrive/Copy of Epinions/Data-final/user_ratings_unbiased_train.txt")
#print(train.shape)


#train = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/testData_Final/ratings_data_train.csv")
train = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/trainImplicit.csv")
train = train[["userId","productId","rating"]]
train
#train = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data_train.csv")

userId  productId  rating
0          692      38404       1
1          850       6956       1
2          245      17754       1
3         1084      18155       1
4         1401      40830       1
...        ...        ...     ...
181908    2041      63223       1
181909     584      36373       1
181910      45       4765       1
181911    6270      47274       1
181912    2658      68970       1

[181913 rows x 3 columns]

In [ ]:
import torch
import torch.utils.data as data
import torch.optim as optim

In [ ]:
rating_mat = list_to_dict(train.values)

In [ ]:
train_dataset = implicit_CF_dataset(7373, 91091, rating_mat, list(train.values), 1)

1819130


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BPR(nn.Module):
	def __init__(self, user_count, item_count, dim, gpu):
		"""
		Parameters
		----------
		user_count : int
		item_count : int
		dim : int
			embedding dimension
		gpu : if available
		"""
		super(BPR, self).__init__()
		self.user_count = user_count
		self.item_count = item_count

		self.user_list = torch.LongTensor([i for i in range(user_count)])
		self.item_list = torch.LongTensor([i for i in range(item_count)])

		if gpu != None:
			self.user_list = self.user_list.to(gpu)
			self.item_list = self.item_list.to(gpu)

		# User / Item Embedding
		self.user_emb = nn.Embedding(self.user_count, dim)
		self.item_emb = nn.Embedding(self.item_count, dim)

		nn.init.normal_(self.user_emb.weight, mean=0., std= 0.01)
		nn.init.normal_(self.item_emb.weight, mean=0., std= 0.01)

		# user-item similarity type
		self.sim_type = 'inner product'
		
		
	def forward(self, batch_user, batch_pos_item, batch_neg_item):
		"""
		Parameters
		----------
		batch_user : 1-D LongTensor (batch_size)
		batch_pos_item : 1-D LongTensor (batch_size)
		batch_neg_item : 1-D LongTensor (batch_size)
		Returns
		-------
		output : 
			Model output to calculate its loss function
		"""
		
		u = self.user_emb(batch_user)			
		i = self.item_emb(batch_pos_item)		
		j = self.item_emb(batch_neg_item)		
		
		pos_score = (u * i).sum(dim=1, keepdim=True)
		neg_score = (u * j).sum(dim=1, keepdim=True)

		output = (pos_score, neg_score)

		return output


	def get_loss(self, output):
		"""Compute the loss function with the model output
		Parameters
		----------
		output : 
			model output (results of forward function)
		Returns
		-------
		loss : float
		"""
		pos_score, neg_score = output[0], output[1]
		loss = -(pos_score - neg_score).sigmoid().log().sum()
		
		return loss


	def forward_multi_items(self, batch_user, batch_items):
		"""forward when we have multiple items for a user,
			Usually for evaluation purpose
		Parameters
		----------
		batch_user : 1-D LongTensor (batch_size)
		batch_items : 2-D LongTensor (batch_size x k)
		Returns
		-------
		score : 2-D FloatTensor (batch_size x k)
		"""

		batch_user = batch_user.unsqueeze(-1)
		batch_user = torch.cat(batch_items.size(1) * [batch_user], 1)
			
		u = self.user_emb(batch_user)		# batch_size x k x dim
		i = self.item_emb(batch_items)		# batch_size x k x dim
		
		score = (u * i).sum(dim=-1, keepdim=False)	
		
		return score



	def get_embedding(self):
		"""get total embedding of users and items
		Returns
		-------
		users : 2-D FloatTensor (num. users x dim)
		items : 2-D FloatTensor (num. items x dim)
		"""
		users = self.user_emb(self.user_list)
		items = self.item_emb(self.item_list)

		return users, items

In [ ]:
model = BPR(7373, 91091, 64, "cpu")
#model = model.cuda()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train_loader = data.DataLoader(train_dataset, batch_size=1000, shuffle=True)
breakout = 0
min_loss = 99999
for epoch in range(0,500):
  epoch_loss = []
  c=0
  for batch_user, batch_pos_item, batch_neg_item in train_loader:
      c+=1
      # Convert numpy arrays to torch tensors
      #batch_user = batch_user.to("cuda")
      #batch_pos_item = batch_pos_item.to("cuda")
      #batch_neg_item = batch_neg_item.to("cuda")
      
      # Forward Pass
      model.train()
      output = model(batch_user, batch_pos_item, batch_neg_item)
      batch_loss = model.get_loss(output)
      epoch_loss.append(batch_loss)
      
      # Backward and optimize
      optimizer.zero_grad()
      batch_loss.backward()
      optimizer.step()	
  epoch_loss = float(torch.mean(torch.stack(epoch_loss)))
  if(round(epoch_loss,2)<round(min_loss,2)):
    print(epoch_loss)
    print(min_loss)
    min_loss = epoch_loss
    breakout = 0 
  else:
    breakout+=1
  if(breakout==3):
    break
  if(epoch%10==0):
      file = "/content/drive/MyDrive/Implicit/Ciao/Models/BPR_"+str(epoch)+"_"+str(1e-3)
      torch.save(model.state_dict(),f=file)
      print("Model Saved")
  print("Train prediction loss is ", round(epoch_loss,2)," for epoch ",epoch)

692.731689453125
99999
Model Saved
Train prediction loss is  692.73  for epoch  0
669.9724731445312
692.731689453125
Train prediction loss is  669.97  for epoch  1
551.0765991210938
669.9724731445312
Train prediction loss is  551.08  for epoch  2
381.9842529296875
551.0765991210938
Train prediction loss is  381.98  for epoch  3
253.70404052734375
381.9842529296875
Train prediction loss is  253.7  for epoch  4
170.32923889160156
253.70404052734375
Train prediction loss is  170.33  for epoch  5
116.56002044677734
170.32923889160156
Train prediction loss is  116.56  for epoch  6
81.5490951538086
116.56002044677734
Train prediction loss is  81.55  for epoch  7
58.431373596191406
81.5490951538086
Train prediction loss is  58.43  for epoch  8
42.903133392333984
58.431373596191406
Train prediction loss is  42.9  for epoch  9
32.237091064453125
42.903133392333984
Model Saved
Train prediction loss is  32.24  for epoch  10
24.743335723876953
32.237091064453125
Train prediction loss is  24.74  fo

KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Ciao/Models/BPR_"+str(40)+"_"+str(1e-3),map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
u_vec,i_vec = model.get_embedding()
print(u_vec.shape)
print(i_vec.shape)

torch.Size([7373, 64])
torch.Size([91091, 64])


In [ ]:
import os
import pandas as pd
import math
import random
import warnings
import numpy as np
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return test_in_top_k['user'].nunique() / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/NCF_80_0.001"))
path = "/content/drive/MyDrive/Implicit/Epinions/testImplicit.csv"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
loss_fn = torch.nn.BCELoss()
loss2_fn = torch.nn.L1Loss()
test = pd.read_csv(path)
test = test[["userId","productId","rating"]]
test_negatives = _sample_negative(list(test["productId"].unique()),test)
a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
del test_negatives
test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
for row in a.itertuples():
  test_users.append(int(row.userId))
  test_items.append(int(row.productId))
  ratings.append(float(row.rating))
  for i in range(len(row.negative_samples)):
      negative_users.append(int(row.userId))
      negative_items.append(int(row.negative_samples[i]))
      negative_ratings.append(float(0))  # negative samples get 0 rating
tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
predr = []
neg_predr = []
for i in range(len(tu)):
  predr.append(torch.sigmoid(torch.matmul(u_vec[tu[i]],i_vec[ti[i]].T)).cpu().detach().numpy())
for i in range(len(nu)):
  neg_predr.append(torch.sigmoid(torch.matmul(u_vec[nu[i]],i_vec[ni[i]].T)).cpu().detach().numpy())
subjects = [tu,ti,predr,nu,ni,neg_predr]
test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
# the golden set
test = pd.DataFrame({'user': test_users,
                    'test_item': test_items,
                    'test_score': test_scores})
# the full set
full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                  'item':  np.append(neg_items,test_items),
                  'score':  np.append(neg_scores,test_scores)})
full = pd.merge(full, test, on=['user'], how='left')
# rank the items according to the scores for each user
full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
full.sort_values(['user', 'rank'], inplace=True)
subjects = full
print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))

Hit Ratio is:  0.49
NDCG Value is:  0.36


In [ ]:
top_k = full[full['rank']<=5]
top_k

user   item     score  test_item  test_score  rank
54310433      0   2947  0.999996         13   0.9905916   1.0
54310434      0   2947  0.999996         53   0.3458226   2.0
54310435      0   2947  0.999996         14    0.989169   3.0
54310436      0   2947  0.999996        151  0.97549987   4.0
54310437      0   2947  0.999996         67   0.3999676   5.0
...         ...    ...       ...        ...         ...   ...
61429837  37951   2947  1.000000     101191  0.86532587   1.0
61429838  37951   2947  1.000000       2947         1.0   2.0
61429839  37951   2947  1.000000      50479   0.8865112   3.0
52990187  37951  14106  0.999817     101191  0.86532587   4.0
52990188  37951  14106  0.999817       2947         1.0   5.0

[122295 rows x 6 columns]

In [ ]:
train_popularity = np.loadtxt("/content/drive/MyDrive/Implicit/Epinions/Data/train_popularity_1.txt")
x = top_k.groupby("user")
users = list(x.groups.keys())
pops = []
for i in users:
  a = x.get_group(i)
  items = a['item'].values
  for j in items:
    s= train_popularity[j]
    pops.append(s/5)
print(sum(pops)/len(pops))

17.805507992965737


In [ ]:
import os
import pandas as pd
import math
import random
import warnings
import numpy as np
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return test_in_top_k['user'].nunique() / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/NCF_80_0.001"))
path = "/content/drive/MyDrive/Implicit/Ciao/"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
files = os.listdir(path)
loss_fn = torch.nn.BCELoss()
loss2_fn = torch.nn.L1Loss()
for f in files:
  if("test" in f):
    if("test" in f and f in ["ratings_data_test_implicit1_1_0.csv","ratings_data_test_implicit1_2_0.csv","ratings_data_test_implicit1_5_0.csv","ratings_data_test_implicit1_10_0.csv"]):
    #if(f == "ratings_data_test_implicit1_10_0.csv" ):
      if(f=="test.csv"):
        continue
      print(f)  
      test = pd.read_csv(path+f)
      test = test[["userId","productId","rating"]]
      test_negatives = _sample_negative(list(test["productId"].unique()),test)
      a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
      del test_negatives
      test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
      for row in a.itertuples():
        test_users.append(int(row.userId))
        test_items.append(int(row.productId))
        ratings.append(float(row.rating))
        for i in range(len(row.negative_samples)):
            negative_users.append(int(row.userId))
            negative_items.append(int(row.negative_samples[i]))
            negative_ratings.append(float(0))  # negative samples get 0 rating
      tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
      predr = []
      neg_predr = []
      for i in range(len(tu)):
        predr.append(torch.sigmoid(torch.matmul(u_vec[tu[i]],i_vec[ti[i]].T)).cpu().detach().numpy())
      for i in range(len(nu)):
        neg_predr.append(torch.sigmoid(torch.matmul(u_vec[nu[i]],i_vec[ni[i]].T)).cpu().detach().numpy())
      subjects = [tu,ti,predr,nu,ni,neg_predr]
      test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
      neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
      # the golden set
      test = pd.DataFrame({'user': test_users,
                          'test_item': test_items,
                          'test_score': test_scores})
      # the full set
      full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                        'item':  np.append(neg_items,test_items),
                        'score':  np.append(neg_scores,test_scores)})
      full = pd.merge(full, test, on=['user'], how='left')
      # rank the items according to the scores for each user
      full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
      full.sort_values(['user', 'rank'], inplace=True)
      subjects = full
      print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
      print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))

ratings_data_test_implicit1_2_0.csv
Hit Ratio is:  0.23
NDCG Value is:  0.14
ratings_data_test_implicit1_5_0.csv
Hit Ratio is:  0.83
NDCG Value is:  0.74
ratings_data_test_implicit1_10_0.csv
Hit Ratio is:  0.96
NDCG Value is:  0.95
ratings_data_test_implicit1_1_0.csv
Hit Ratio is:  0.13
NDCG Value is:  0.08
